## Phase 1: Data extraction

https://news.ycombinator.com/item?id=41614126

https://github.com/conjuncts/gmft

https://www.reddit.com/r/LocalLLaMA/comments/1854d06/table_extraction_from_pdf/ ==> some guy did https://docs.google.com/spreadsheets/d/12IhxHZbYF71dPl32PQpF_6pg9e9S8f9W4sTHt-B0KTg/edit?gid=0#gid=0 


https://www.getmagical.com/blog/extract-table-from-pdf


As to whether traditional OCR or LLMs are better for this task

#### Conclusions:
- We REALLY want to keep the right figures, so potentially trying lower end models for this, while it can be accurate a fair amount of the time, likely does not counteract the risk of it eventually hallucinating. Not all LLMs are equal.
- [Marker](https://github.com/vikParuchuri/marker) has 20k stars on github and was found from the excel sheet; but has a commercial usage agreement so Nahh

https://github.com/conjuncts/gmft ? => Relies on Table transformers (microsoft), small, seems ideal. I've also seen stuff for AWS textract but let's keep it simple for now.

In [1]:
import gmft
from gmft.auto import AutoTableDetector, AutoTableFormatter
from gmft.pdf_bindings import PyPDFium2Document
import pandas as pd

# Load PDF and extract tables
pdf_path = "./pdf_inputs/fwc_sample_financial_statement 1.pdf"
detector = AutoTableDetector()
formatter = AutoTableFormatter()

doc = PyPDFium2Document(pdf_path)
tables = []

for page_num, page in enumerate(doc):
    extracted_tables = detector.extract(page)
    for table_index, table in enumerate(extracted_tables):
        formatted_table = formatter.extract(table)
        table_data = formatted_table.df()
        if table_data is not None:
            tables.append(table_data)

doc.close()

# Display extracted tables
if tables:
    for i, table_df in enumerate(tables):
        ace_tools.display_dataframe_to_user(name=f"Extracted Table {i+1}", dataframe=table_df)
else:
    print("No tables were extracted.")


NameError: name 'ace_tools' is not defined

> Tried gmft and it has all these nice metrics etc but when I actually used it, it was terrible - because after some diving in it seems to be designed for actual tables in scientific journals etc, whilst this pdf is amajority of white space, tables go over multiple pages etc so it really does not deal well with this.

Would be way better to just get a text reader for this case

In [18]:
import PyPDF2
pdf_path = "fwc_sample_financial_statement 1.pdf"


def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
            text += "\n==================\n"
    return text

pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)

   
FS 023 Sample financial statement   10 March  2023  | p. 1 Fact Sheet  FS 023 | 10 March  2023  
 
Sam ple financial statements  
 
Statement of  comprehensive income  
For the year ended 30 June  
  
Notes  Last Year  
$ Previous Year  
$ 
Revenue     
Membership subscriptions   6,748,000  6,571,000  
Interest   251,000  231,000  
Rental income   185,000  244,000  
Other revenue  3A 613,000  655,000  
Total revenue   7,797,000  7,701,000  
    
Expenses     
Employee expenses  4A 3,167,000  3,023,000  
Capitation fees  4B 830,000  823,000  
Affiliation fees  4C 228,000  231,000  
Depreciation  4D 97,000  88,000  
Administration expenses  4E 1,649,000  1,475,000  
Legal costs  4F 296,000  62,000  
Donations  4G 77,000  120,000  
Motor vehicle expenses  4H 89,000  79,000  

   
FS 023 Sample financial statement   10 March  2023  | p. 2 Other expenses  4I 835,000  775,000  
Total expenses   7,268,000  6,676,000  
    
Surplus (deficit) for the year   529,000  1,025,000  
    
Other c

In [1]:
import pdfplumber # pdfplumber IS SLOWER BTW by a fair bit, but it also does give much nicer formatting.
import pandas as pd
import re

pdf_path = "fwc_sample_financial_statement 1.pdf"

def extract_pdf_by_pages(pdf_path):
    pages_text = {}
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, 1):
            pages_text[f"page_{page_num}"] = page.extract_text()
    return pages_text

# Extract all pages
pages = extract_pdf_by_pages(pdf_path)

# Print each page to see what we're working with
for page_num, text in pages.items():
    print(f"\n====== {page_num} ======\n")
    print(text)


====== page_1 ======

Fact Sheet FS 023 | 10 March 2023
Sample financial statements
Statement of comprehensive income
For the year ended 30 June
Last Year Previous Year
Notes $ $
Revenue
Membership subscriptions 6,748,000 6,571,000
Interest 251,000 231,000
Rental income 185,000 244,000
Other revenue 3A 613,000 655,000
Total revenue 7,797,000 7,701,000
Expenses
Employee expenses 4A 3,167,000 3,023,000
Capitation fees 4B 830,000 823,000
Affiliation fees 4C 228,000 231,000
Depreciation 4D 97,000 88,000
Administration expenses 4E 1,649,000 1,475,000
Legal costs 4F 296,000 62,000
Donations 4G 77,000 120,000
Motor vehicle expenses 4H 89,000 79,000
FS 023 Sample financial statement 10 March 2023 | p. 1

====== page_2 ======

Other expenses 4I 835,000 775,000
Total expenses 7,268,000 6,676,000
Surplus (deficit) for the year 529,000 1,025,000
Other comprehensive income
Gain on revaluation – investment property - 342,000
Total comprehensive income for the year 529,000 1,367,000
The above statem

#### Conclusions
PDF plumber takes (noticeably) longer, but is more accurate/has less missing characters so we'll be using this because ideally, the data extraction script should only run once (and we can save the outputs to a DB or something eventually... (foreshadowing: RAG yay!))

In [13]:
def clean_text(text):
    # Remove page headers and footers
    text = re.sub(r'FS 023 Sample financial statement.*?\| p\. \d+', '', text)
    text = re.sub(r'The above statement should be read in conjunction with the notes\.', '', text)
    text = re.sub(r'© Commonwealth of Australia.*?legal advice\.', '', text, flags=re.DOTALL)
    return text.strip()

# Clean each page
cleaned_pages = {page_num: clean_text(text) for page_num, text in pages.items()}

# Look at cleaned text for first page
print(cleaned_pages['page_1'], cleaned_pages['page_2'])

Fact Sheet FS 023 | 10 March 2023
Sample financial statements
Statement of comprehensive income
For the year ended 30 June
Last Year Previous Year
Notes $ $
Revenue
Membership subscriptions 6,748,000 6,571,000
Interest 251,000 231,000
Rental income 185,000 244,000
Other revenue 3A 613,000 655,000
Total revenue 7,797,000 7,701,000
Expenses
Employee expenses 4A 3,167,000 3,023,000
Capitation fees 4B 830,000 823,000
Affiliation fees 4C 228,000 231,000
Depreciation 4D 97,000 88,000
Administration expenses 4E 1,649,000 1,475,000
Legal costs 4F 296,000 62,000
Donations 4G 77,000 120,000
Motor vehicle expenses 4H 89,000 79,000 Other expenses 4I 835,000 775,000
Total expenses 7,268,000 6,676,000
Surplus (deficit) for the year 529,000 1,025,000
Other comprehensive income
Gain on revaluation – investment property - 342,000
Total comprehensive income for the year 529,000 1,367,000


In [12]:
def extract_income_statement(cleaned_pages):
    # Combine pages 1 and 2 which contain the income statement
    text = cleaned_pages['page_1'] + '\n' + cleaned_pages['page_2']
    
    lines = []
    for line in text.split('\n'):
        # Look for lines with amounts
        match = re.search(r'([A-Za-z\s–]+)\s+(?:\d[A-Z])?\s*([\d,]+|-)\s+([\d,]+|-)', line)
        if match:
            item = match.group(1).strip()
            last_year = match.group(2).replace(',', '')
            prev_year = match.group(3).replace(',', '')
            
            # Convert to numeric, handling '-' as 0
            last_year = 0 if last_year == '-' else float(last_year)
            prev_year = 0 if prev_year == '-' else float(prev_year)
            
            lines.append({
                'Item': item,
                'Last Year': last_year,
                'Previous Year': prev_year
            })
    
    df = pd.DataFrame(lines)
    return df

income_statement_df = extract_income_statement(cleaned_pages)
print("Income Statement:")
display(income_statement_df)  # Jupyter will show this as a nice table

Income Statement:


,Item,Last Year,Previous Year
0,Membership subscriptions,6748000.0,6571000.0
1,Interest,251000.0,231000.0
2,Rental income,185000.0,244000.0
3,Other revenue,613000.0,655000.0
4,Total revenue,7797000.0,7701000.0
5,Employee expenses,3167000.0,3023000.0
6,Capitation fees,830000.0,823000.0
7,Affiliation fees,228000.0,231000.0
8,Depreciation,97000.0,88000.0
9,Administration expenses,1649000.0,1475000.0


In [ ]:
import pandas as pd # Sideventure of trying to just throw the pdf at gemini and telling it to generate a csv (DID NOT WORK AT ALL)

# Replace 'your_file.csv' with the actual name of your CSV file
df = pd.read_csv('gemini_test.csv') # wow gemini CANNOT make csvs LOL okay looks like this manual reading off pdf is better atleast.

# Now you can work with the data in the DataFrame 'df'
print(df)

                                                Statement  Item   Category  \
Income                                            Revenue   NaN        NaN   
NaN                              Membership subscriptions   NaN  6748000.0   
NaN                                              Interest   NaN   251000.0   
NaN                                         Rental income   NaN   185000.0   
NaN                                         Other revenue   NaN   613000.0   
...                                                   ...   ...        ...   
NaN                                       Total Cash Used   NaN        NaN   
NaN                    NET CASH FROM FINANCING ACTIVITIES   NaN  8000000.0   
NaN                  NET INCREASE/(DECREASE) IN CASH HELD   NaN  1862000.0   
NaN     Cash and cash equivalents at the beginning of ...   NaN  4955000.0   
NaN     Cash and cash equivalents at the end of the re...   NaN  3093000.0   

        Last Year  Previous Year  
Income        NaN           

In [ ]:
    """
    
    What I wanna do is probably... Hmm. Okay, so let's clean the data up first (get rid of useless junk), 
    """

## Phase 5 - Scaling And Stuff
